In [1]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import numpy as np
import matplotlib.pyplot as pyplot
%matplotlib inline  
import pandas as pd
import scipy.sparse as sps

from Data_manager.Movielens.Movielens10MReader import Movielens10MReader
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


In [2]:
def load_URM(file_path):

    data = pd.read_csv(file_path)


    user_list = data['row'].tolist()
    item_list = data['col'].tolist()
    rating_list = data['data'].tolist()

    return sps.coo_matrix((rating_list, (user_list, item_list))).tocsr()

In [3]:
def load_ICM(file_path):

    data = pd.read_csv(file_path)

    item_list = data['row'].tolist()
    attribute_list = data['col'].tolist()
    rating_list = data['data'].tolist()

    return sps.coo_matrix((rating_list, (item_list, attribute_list))).tocsr()

In [4]:
URM_all = load_URM('input/data_train.csv')
ICM_genres = load_ICM('input/data_ICM_genre.csv')

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13639 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13640 ( 0.1%) Users that have less than 1 test interactions


In [ ]:
# In order to evaluate put it in a recommender class
from Recommenders import BaseRecommender
from lightfm import LightFM
import numpy as np

class LightFMWrapper(BaseRecommender):
    """LightFMWrapper"""

    RECOMMENDER_NAME = "LightFMWrapper"

    def __init__(self, URM_train):
        super(LightFMWrapper, self).__init__(URM_train)


    def fit(self, ITEM_ALPHA, NUM_COMPONENTS, NUM_EPOCHS, NUM_THREADS):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss='warp',
                                     item_alpha=ITEM_ALPHA,
                                     no_components=NUM_COMPONENTS)

        self.lightFM_model = self.lightFM_model.fit(URM_train, 
                                       epochs=NUM_EPOCHS,
                                       num_threads=NUM_THREADS)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items) if items_to_compute is None else np.array(items_to_compute)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores
